In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

import warnings
warnings.filterwarnings('ignore')

In [235]:
import re
from konlpy.tag import Okt
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [236]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\medici\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
#플레이 리스트
train = pd.read_json('data/train.json')
train['tag__'] = train['tags'].apply(lambda x: (" ").join(x))


#곡 정보
song_meta = pd.read_json('data/song_meta.json')
song_meta['artist_name_basket__'] = song_meta['artist_name_basket'].apply(lambda x: (" ").join(x))
song_meta['song_gn_gnr_basket__'] = song_meta['song_gn_gnr_basket'].apply(lambda x: (" ").join(x))

In [266]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name_basket__,song_gn_gnr_basket__
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,Various Artists,GN0900
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,Murray Perahia,GN1600
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,Peter Gabriel,GN0900
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,Matoma,GN1100
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,Jude Law,GN1800
...,...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,Fela Kuti,GN2000
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,Cyndi Lauper,GN0900
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,윤종신,GN0100
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,Nature Piano,GN1800


In [265]:
#텍스트 정제 (특수기호 제거)
txt = []

for i, document in enumerate(train['plylst_title']):
    document = re.sub(r'[^ A-Za-z0-9가-힣]', '', document) #특수기호 제거, 정규 표현식    
    txt.append(document.strip()) # 문장 왼쪽 오른쪽에 있는 공백 제거
    
train['playlist_title_txt'] = txt
train

,tags,id,plylst_title,songs,like_cnt,updt_date,tag__,playlist_title_txt
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,락,여행같은 음악
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,추억 회상,요즘 너 말야
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,까페 잔잔한,편하게 잔잔하게 들을 수 있는 곡
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송,크리스마스 분위기에 흠뻑 취하고 싶을때
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,댄스,추억의 노래
...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,록메탈 밴드사운드 록 락메탈 메탈 락 extreme,METAL ESM 2
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,일렉,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,담시 가족 눈물 그리움 주인공 나의_이야기 사랑 친구,1 눈물이 앞을 가리는 나의이야기
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,잔잔한 버스 퇴근버스 Pop 풍경 퇴근길,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP


In [35]:
txt = []
for i, document in enumerate(train['tag__']):
    document = re.sub(r'[^가-힣]', ' ', document) #특수기호 제거, 정규 표현식    
    txt.append(document.strip()) # 문장 왼쪽 오른쪽에 있는 공백 제거
    
txt

['락',
 '추억 회상',
 '까페 잔잔한',
 '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
 '댄스',
 '운동 드라이브     트로피컬하우스 힐링 기분전환      팝 트렌드 일렉',
 '짝사랑 취향저격 슬픔 고백 사랑 이별',
 '잔잔한 추억 회상',
 '일렉트로니카 포크 메탈 락 댄스 인디',
 '록       이일우  에센셜 메탈      락',
 '댄스 걸그룹댄스 스트레스해소',
 '새해 여행 프로필음악 카카오톡 기분전환 소원 프로필 소망 다짐 카톡',
 '듣고 우울 힐링 이거 힘내',
 '힙합 느낌있는 밤 새벽     감각적인 드라이브 국내 그루브한',
 '가을 재즈',
 '락',
 '감성 질리지않는 나만알고싶은',
 '봄 설렘 사랑',
 '비오는날 누군가생각날때',
 '스밍 목록 폐막식 올림픽 엑소',
 '조용히 혼자 또는 새벽감성 고민 맥주한잔',
 '카페 재즈 잔잔한',
 '',
 '댄스',
 '감성 어쿠스틱 잔잔한 새벽 편안한 인디 밤',
 '잔잔한 밤 새벽',
 '힐링 드라이브 에너지 인디 여행',
 '듀엣 취향저격 달달 피쳐링 사랑 남녀',
 '까페 설렘 사랑',
 '힙합 신인 국힙 인디 신나는',
 '거슈윈 존윌리엄스 랩소디인블루 미국클래식 영화클래식 아메리칸클래식',
 '겨울 기분전환',
 '아침 빅밴드 상쾌한 재즈 밝은 스윙 스윙재즈',
 '휴식 힐링 설렘 사랑',
 '여름 일렉트로니카 매장음악 취향저격 댄스 드라이브 여행 기분전환 일렉 신나는',
 '까페 설렘 사랑',
 '잔잔한 설렘 사랑',
 '헬스 스포츠 피트니스 운동 다이어트 런닝 레깅스 필라테스 산책 요가',
 '발라드',
 '겨울 크리스마스 캐롤',
 '위로 힐링',
 '기분전환 가을',
 '년대 감성   년대발라드 올드송 추억 휴식   년대 옛날노래',
 '월드 연주 재즈',
 '잔잔한 밤 새벽',
 '슬픔 밤 이별 새벽',
 '클럽 스트레스',
 '휴식 주말 오후 나른한          편안한',
 '힐링 휴식 운동

In [36]:
txt = []
for i, document in enumerate(train['tag__']):
    document = re.sub(r'[^A-Za-z가-힣0-9]', ' ', document) #특수기호 제거, 정규 표현식    
    txt.append(document.strip()) # 문장 왼쪽 오른쪽에 있는 공백 제거
    
txt

['락',
 '추억 회상',
 '까페 잔잔한',
 '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
 '댄스',
 '운동 드라이브 Pop 트로피컬하우스 힐링 기분전환 2017 팝 트렌드 일렉',
 '짝사랑 취향저격 슬픔 고백 사랑 이별',
 '잔잔한 추억 회상',
 '일렉트로니카 포크 메탈 락 댄스 인디',
 '록 Metal 이일우 M에센셜 메탈 Rock 락',
 'kpop 댄스 걸그룹댄스 스트레스해소',
 '새해 여행 프로필음악 카카오톡 기분전환 소원 프로필 소망 다짐 카톡',
 '듣고 우울 힐링 이거 힘내',
 '힙합 느낌있는 밤 새벽 RnB 감각적인 드라이브 국내 그루브한',
 '가을 재즈',
 '락',
 '감성 질리지않는 나만알고싶은 Pop',
 '봄 설렘 사랑',
 '비오는날 누군가생각날때',
 '스밍 목록 폐막식 올림픽 엑소',
 '조용히 혼자 또는 새벽감성 고민 맥주한잔',
 '카페 재즈 잔잔한',
 'OST',
 '댄스',
 '감성 어쿠스틱 잔잔한 새벽 편안한 인디 밤',
 '잔잔한 밤 새벽',
 '힐링 드라이브 에너지 인디 여행',
 '듀엣 취향저격 달달 피쳐링 사랑 남녀',
 '까페 설렘 사랑',
 '힙합 신인 국힙 인디 신나는',
 '거슈윈 존윌리엄스 랩소디인블루 미국클래식 영화클래식 아메리칸클래식',
 '겨울 기분전환',
 '아침 빅밴드 상쾌한 재즈 밝은 스윙 스윙재즈',
 '휴식 힐링 설렘 사랑',
 '여름 일렉트로니카 매장음악 취향저격 댄스 드라이브 여행 기분전환 일렉 신나는',
 '까페 설렘 사랑',
 '잔잔한 설렘 사랑',
 '헬스 스포츠 피트니스 운동 다이어트 런닝 레깅스 필라테스 산책 요가',
 '발라드',
 '겨울 크리스마스 캐롤',
 '위로 힐링',
 '기분전환 가을',
 '80년대 감성 90년대발라드 올드송 추억 휴식 90년대 옛날노래',
 '월드 연주 재즈',
 '잔잔한 밤 새벽',
 '슬픔 밤 이별 새벽',
 '클럽 스트레스',
 '휴식 주말 오후 나른한 Chillout 편안

In [25]:
def korean_preprocessing(txt, okt, removes_stopwords = False, stop_words = []):
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 사용하지 않고 미리 생성한 후 인자로 받는다
    # 불용어 사전은 사용자가 직접 입력해야함. 기본값은 빈 리스트
    
    
    txt = re.sub('[^가-힣0-9]',"",txt)
    # 한글과 숫자만 나오게 함
    
    
    # okt 객체를 활용해 형태소 단위로 나눈다
    word_txt = okt.morphs(txt, stem = True)
    
    if removes_stopwords == True:
        word_txt = [token for token in word_txt if not token in stop_words]
        word_txt = []
    
    return word_txt

In [237]:
def english_preprocessing(txt, remove_stopwords = False): # 불용어 제거는 옵션으로 설정
    
    # html 용어 제거
    txt = BeautifulSoup(txt, 'html5lib').get_text()
    
    # 영어 문자를 제외한 나머지 문자는 모두 공백으로 변환
    txt = re.sub("[^a-zA-Z0-9]",' ',txt)
    
    words = txt.lower().split() # 전부 소문자로 변환 후 단어마다 나눠서 단어리스트로 만든다
    
    # remove_stopwords == True인 경우 불용어 제거
    if remove_stopwords:
        
        stops = set(stopwords.words('english')) # 영어 불용어 불러오기

        words = [w for w in words if not w in stops] # stopwords에 있는 단어 제거
        
        clean_txt = ' '.join(words) # 각 단어들을 띄어쓰기로 묶어줌
    
    else:
        clean_txt = ' '.join(words)
    

    return clean_txt

In [256]:
stopword = pd.read_csv('data/stopword.csv')
stopword = stopword.iloc[:,0].tolist()

In [239]:
text = train['tag__']

In [29]:
# 한글 토큰화
ko_token = [korean_preprocessing(txt, okt, removes_stopwords=True, stop_words=stopword) for txt in text]
ko_token

KeyboardInterrupt: 

In [240]:
# 영어 토큰화
en_token = [english_preprocessing(txt, remove_stopwords=True) for txt in text]

In [71]:
x = pd.read_csv('data/kr_en_token.csv')
x.sample(100)

,0
78678,"['누눈', '이다', '첫', '눈', '이다']"
11420,"['인디씬', '떠오르다', '샛별', '딸기', '737']"
63691,"['여자도', '반하다', '매력', '가득하다', '멋지다', '언니', '노래'..."
113567,"['봄', '음악', 'Spring', 'Music']"
39688,[]
80704,"['Just', 'CHRISTMAS']"
102976,"['감성', '충만', '카페', '듣기', '좋다', '노래']"
21884,"['더욱', '아름답다', '듀엣곡']"
102502,"['일본', '남자', '보컬', '밴드']"
49831,"['은', '쓸쓸하다', '발렌타인']"


In [73]:
def total_preprocessing(txt, okt, removes_stopwords = False, stop_words = []):
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 사용하지 않고 미리 생성한 후 인자로 받는다
    # 불용어 사전은 사용자가 직접 입력해야함. 기본값은 빈 리스트
    
    
    txt = re.sub('[^A-Za-z가-힣0-9]'," ",txt)
        
    
    # okt 객체를 활용해 형태소 단위로 나눈다
    word_txt = okt.morphs(txt, stem = True)
    
    if removes_stopwords == True:
        word_txt = [token for token in word_txt if not token in stop_words]    

        stops = set(stopwords.words('english')) # 영어 불용어 불러오기

        word_txt = [w for w in word_txt if not w in stops] # stopwords에 있는 단어 제거
    
    return word_txt 

In [77]:
text = train['tag__']

In [78]:
x = [total_preprocessing(txt, okt, removes_stopwords=False, stop_words=stopword)for txt in text]
x

[['락'],
 ['추억', '회상'],
 ['까페', '잔잔하다'],
 ['연말',
  '눈',
  '오는',
  '날',
  '캐럴',
  '분위기',
  '따다',
  '한',
  '크리스마스캐럴',
  '겨울',
  '노래',
  '크리스마스',
  '겨울왕국',
  '크리스마스',
  '송'],
 ['댄스'],
 ['운동',
  '드라이브',
  'Pop',
  '트로피',
  '컬',
  '하우스',
  '힐링',
  '기분',
  '전환',
  '2017',
  '팝',
  '트렌드',
  '일',
  '렉'],
 ['짝사랑', '취향', '저격', '슬픔', '고백', '사랑', '이별'],
 ['잔잔하다', '추억', '회상'],
 ['일렉트로니카', '포크', '메탈', '락', '댄스', '이다'],
 ['록', 'Metal', '이일우', 'M', '에센셜', '메탈', 'Rock', '락'],
 ['kpop', '댄스', '걸그룹댄스', '스트레스', '해소'],
 ['새해',
  '여행',
  '프로필',
  '음악',
  '카카오',
  '톡',
  '기분',
  '전환',
  '소원',
  '프로필',
  '소망',
  '다짐',
  '카톡'],
 ['듣다', '우울', '힐링', '이', '거', '힘내다'],
 ['힙합', '느끼다', '밤', '새벽', 'RnB', '감각', '적', '인', '드라이브', '국내', '그루브', '한'],
 ['가을', '재즈'],
 ['락'],
 ['감성', '질리', '지', '않다', '나', '만', '알다', 'Pop'],
 ['봄', '설레다', '사랑'],
 ['비', '오는', '날', '누군가', '생각나다', '때'],
 ['스밍', '목록', '폐막식', '올림픽', '엑소'],
 ['조용하다', '혼자', '또는', '새벽', '감성', '고민', '맥주', '한잔'],
 ['카페', '재즈', '잔잔하다'],
 ['OST'],
 ['댄스'],
 ['감성', '어쿠스틱'

In [81]:
def total_preprocessing_noun(txt, okt, removes_stopwords = False, stop_words = []):
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 사용하지 않고 미리 생성한 후 인자로 받는다
    # 불용어 사전은 사용자가 직접 입력해야함. 기본값은 빈 리스트
    
    
    txt = re.sub('[^A-Za-z가-힣0-9]'," ",txt)
        
    
    # okt 객체를 활용해  명사만 추출
    word_txt = okt.nouns(txt)
    
    if removes_stopwords == True:
        word_txt = [token for token in word_txt if not token in stop_words]    

        stops = set(stopwords.words('english')) # 영어 불용어 불러오기

        word_txt = [w for w in word_txt if not w in stops] # stopwords에 있는 단어 제거
    
    return word_txt 

In [82]:
y = [total_preprocessing_noun(txt, okt, removes_stopwords=False, stop_words=stopword)for txt in text]
y

[['락'],
 ['추억', '회상'],
 ['까페'],
 ['연말',
  '눈',
  '날',
  '캐럴',
  '분위기',
  '크리스마스캐럴',
  '겨울',
  '노래',
  '크리스마스',
  '겨울왕국',
  '크리스마스',
  '송'],
 ['댄스'],
 ['운동', '드라이브', '트로피', '컬', '하우스', '힐링', '기분', '전환', '팝', '트렌드', '렉'],
 ['짝사랑', '취향', '저격', '슬픔', '고백', '사랑', '이별'],
 ['추억', '회상'],
 ['일렉트로니카', '포크', '메탈', '락', '댄스'],
 ['록', '이일우', '에센셜', '메탈', '락'],
 ['댄스', '걸그룹댄스', '스트레스', '해소'],
 ['새해',
  '여행',
  '프로필',
  '음악',
  '카카오',
  '톡',
  '기분',
  '전환',
  '소원',
  '프로필',
  '소망',
  '다짐',
  '카톡'],
 ['우울', '힐링', '거'],
 ['힙합', '밤', '새벽', '감각', '드라이브', '국내', '그루브'],
 ['가을', '재즈'],
 ['락'],
 ['감성', '질리', '나'],
 ['봄', '사랑'],
 ['비', '날', '누군가', '때'],
 ['스밍', '목록', '폐막식', '올림픽', '엑소'],
 ['혼자', '새벽', '감성', '고민', '맥주', '한잔'],
 ['카페', '재즈'],
 [],
 ['댄스'],
 ['감성', '어쿠스틱', '새벽', '밤'],
 ['밤', '새벽'],
 ['힐링', '드라이브', '에너지', '여행'],
 ['듀엣', '취향', '저격', '달달', '피쳐링', '사랑', '남녀'],
 ['까페', '사랑'],
 ['힙합', '신인', '국힙'],
 ['거슈윈', '존', '윌리엄스', '랩소디', '블루', '미국', '클래식', '영화', '클래식', '아메리칸', '클래식'],
 ['겨울', '기분', '전환'],
 ['아침',

---
---
---

In [246]:
def total_preprocessing_part(txt, okt, removes_stopwords = False, stop_words = []):
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 사용하지 않고 미리 생성한 후 인자로 받는다
    # 불용어 사전은 사용자가 직접 입력해야함. 기본값은 빈 리스트
    
    
    txt = re.sub('[^A-Za-z가-힣0-9]'," ",txt)
        
    
    # okt 객체를 활용해  명사와 동사만 추출
    
    sentences_tag = []
    
    morph = okt.pos(txt)
    sentences_tag.append(morph)
    
            
    noun_adj_list = []
    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['Noun', 'Verb', 'Adjective', 'Alpha', 'Number']:
                noun_adj_list.append(word)   
    
    
    if removes_stopwords == True:
        noun_adj_list = [token for token in noun_adj_list if not token in stop_words]    

        stops = set(stopwords.words('english')) # 영어 불용어 불러오기

        noun_adj_list = [w for w in noun_adj_list if not w in stops] # stopwords에 있는 단어 제거
        
        
    
    return noun_adj_list

In [244]:
text = train['tag__'].tolist()
text

['락',
 '추억 회상',
 '까페 잔잔한',
 '연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송',
 '댄스',
 '운동 드라이브 Pop 트로피컬하우스 힐링 기분전환 2017 팝 트렌드 일렉',
 '짝사랑 취향저격 슬픔 고백 사랑 이별',
 '잔잔한 추억 회상',
 '일렉트로니카 포크 메탈 락 댄스 인디',
 '록 Metal 이일우 M에센셜 메탈 Rock 락',
 'kpop 댄스 걸그룹댄스 스트레스해소',
 '새해 여행 프로필음악 카카오톡 기분전환 소원 프로필 소망 다짐 카톡',
 '듣고 우울 힐링 이거 힘내',
 '힙합 느낌있는 밤 새벽 RnB 감각적인 드라이브 국내 그루브한',
 '가을 재즈',
 '락',
 '감성 질리지않는 나만알고싶은 Pop',
 '봄 설렘 사랑',
 '비오는날 누군가생각날때',
 '스밍 목록 폐막식 올림픽 엑소',
 '조용히 혼자 또는 새벽감성 고민 맥주한잔',
 '카페 재즈 잔잔한',
 'OST',
 '댄스',
 '감성 어쿠스틱 잔잔한 새벽 편안한 인디 밤',
 '잔잔한 밤 새벽',
 '힐링 드라이브 에너지 인디 여행',
 '듀엣 취향저격 달달 피쳐링 사랑 남녀',
 '까페 설렘 사랑',
 '힙합 신인 국힙 인디 신나는',
 '거슈윈 존윌리엄스 랩소디인블루 미국클래식 영화클래식 아메리칸클래식',
 '겨울 기분전환',
 '아침 빅밴드 상쾌한 재즈 밝은 스윙 스윙재즈',
 '휴식 힐링 설렘 사랑',
 '여름 일렉트로니카 매장음악 취향저격 댄스 드라이브 여행 기분전환 일렉 신나는',
 '까페 설렘 사랑',
 '잔잔한 설렘 사랑',
 '헬스 스포츠 피트니스 운동 다이어트 런닝 레깅스 필라테스 산책 요가',
 '발라드',
 '겨울 크리스마스 캐롤',
 '위로 힐링',
 '기분전환 가을',
 '80년대 감성 90년대발라드 올드송 추억 휴식 90년대 옛날노래',
 '월드 연주 재즈',
 '잔잔한 밤 새벽',
 '슬픔 밤 이별 새벽',
 '클럽 스트레스',
 '휴식 주말 오후 나른한 Chillout 편안

In [245]:
# 명사 동사 영어 숫자만 뽑은거
# z = [total_preprocessing_part(txt, okt, removes_stopwords=True, stop_words=stopword)for txt in text]
# z

[['락'],
 ['추억', '회상'],
 ['까페', '잔잔한'],
 ['연말',
  '눈',
  '날',
  '캐럴',
  '분위기',
  '따듯',
  '크리스마스캐럴',
  '겨울',
  '노래',
  '크리스마스',
  '겨울왕국',
  '크리스마스',
  '송'],
 ['댄스'],
 ['운동',
  '드라이브',
  'Pop',
  '트로피',
  '컬',
  '하우스',
  '힐링',
  '기분',
  '전환',
  '2017',
  '팝',
  '트렌드',
  '렉'],
 ['짝사랑', '취향', '저격', '슬픔', '고백', '사랑', '이별'],
 ['잔잔한', '추억', '회상'],
 ['일렉트로니카', '포크', '메탈', '락', '댄스', '인디'],
 ['록', 'Metal', '이일우', 'M', '에센셜', '메탈', 'Rock', '락'],
 ['kpop', '댄스', '걸그룹댄스', '스트레스', '해소'],
 ['새해',
  '여행',
  '프로필',
  '음악',
  '카카오',
  '톡',
  '기분',
  '전환',
  '소원',
  '프로필',
  '소망',
  '다짐',
  '카톡'],
 ['듣고', '우울', '힐링', '거', '힘내'],
 ['힙합', '느낌있는', '밤', '새벽', 'RnB', '감각', '드라이브', '국내', '그루브'],
 ['가을', '재즈'],
 ['락'],
 ['감성', '질리', '않는', '알고싶은', 'Pop'],
 ['봄', '설렘', '사랑'],
 ['비', '날', '누군가', '생각날'],
 ['스밍', '목록', '폐막식', '올림픽', '엑소'],
 ['조용히', '새벽', '감성', '고민', '맥주', '한잔'],
 ['카페', '재즈', '잔잔한'],
 ['OST'],
 ['댄스'],
 ['감성', '어쿠스틱', '잔잔한', '새벽', '편안한', '인디', '밤'],
 ['잔잔한', '밤', '새벽'],
 ['힐링', '드라이브', '에너지', '인디', 

In [257]:
stopword = pd.read_csv('data/stopword.csv')
stopword = stopword.iloc[:,0].tolist()

In [263]:
okt.morphs('기리보이의노래는길이보인다')

['기리보이', '의', '노래는길', '이보', '인', '다']

In [264]:
total_preprocessing_part('기리보이의노래는길이보인다', okt, removes_stopwords=True, stop_words=stopword)

['기리보이', '노래는길', '이보']

In [259]:
# 명사 동사 형용사 영어 숫사만 뽑은거 
stopword = pd.read_csv('data/stopword.csv')
stopword = stopword.iloc[:,0].tolist()

a = [total_preprocessing_part(txt, okt, removes_stopwords=True, stop_words=stopword)for txt in text]
a



[['락'],
 ['추억', '회상'],
 ['까페', '잔잔한'],
 ['연말',
  '눈',
  '캐럴',
  '분위기',
  '따듯',
  '크리스마스캐럴',
  '겨울',
  '노래',
  '크리스마스',
  '겨울왕국',
  '크리스마스',
  '송'],
 ['댄스'],
 ['운동',
  '드라이브',
  'Pop',
  '트로피',
  '컬',
  '하우스',
  '힐링',
  '기분',
  '전환',
  '2017',
  '팝',
  '트렌드',
  '렉'],
 ['짝사랑', '취향', '저격', '슬픔', '고백', '사랑', '이별'],
 ['잔잔한', '추억', '회상'],
 ['일렉트로니카', '포크', '메탈', '락', '댄스', '인디'],
 ['록', 'Metal', '이일우', 'M', '에센셜', '메탈', 'Rock', '락'],
 ['kpop', '댄스', '걸그룹댄스', '스트레스', '해소'],
 ['새해',
  '여행',
  '프로필',
  '음악',
  '카카오',
  '톡',
  '기분',
  '전환',
  '소원',
  '프로필',
  '소망',
  '다짐',
  '카톡'],
 ['듣고', '우울', '힐링', '거', '힘내'],
 ['힙합', '느낌있는', '밤', '새벽', 'RnB', '감각', '드라이브', '국내', '그루브'],
 ['가을', '재즈'],
 ['락'],
 ['감성', '질리', '않는', '알고싶은', 'Pop'],
 ['봄', '설렘', '사랑'],
 ['비', '누군가', '생각날'],
 ['스밍', '목록', '폐막식', '올림픽', '엑소'],
 ['조용히', '새벽', '감성', '고민', '맥주', '한잔'],
 ['카페', '재즈', '잔잔한'],
 ['OST'],
 ['댄스'],
 ['감성', '어쿠스틱', '잔잔한', '새벽', '편안한', '인디', '밤'],
 ['잔잔한', '밤', '새벽'],
 ['힐링', '드라이브', '에너지', '인디', '여행'],
 ['듀엣

>8살 >> "8",  "살"   
10대 >> "10",   "대"   
스테디셀러 >> "스", "테디", "셀러"  
도입부 >> '도입', '부'   
볼빨간사춘기 >> '볼', "빨간", "사춘기"   
"날"      
"거"   
"때"    
['우울할',
  '땐',
  '어도',
  '되죠',
  '괜찮은',
  '척',
  '않아도',
  '되요',
  '애써',
  '밝으려고',
  '하지말아요',
  '진짜',
  '나를',
  '찾아가는',
  '과정']   
이런 단어들 처리는 어떻게 해야하나    
날 거 때 같은 경우에는 불용어에 넣어서 처리하면 될 듯하기도 함   


>중복단어는 그냥 내비둬야할까? 아니면 중복값은 제거 해줄까??   

>8살 10대 스테디텔러 이런 단어들을 하나의 단어로 인식하게 할 수 있을까??
Number + "살" 이런 식으로 붙을 수 있게 가능한가?   



>영어랑 한글이랑 한 번에 자연어 처리가 가능할까?

In [260]:
total = []
total.append(a)

In [261]:
total_df = pd.DataFrame(total).T
total_df[1] = total_df[0].apply(lambda x : " ".join(x))
total_df[1] = total_df[1].apply(lambda x : x.lower())
total_df

,0,1
0,[락],락
1,"[추억, 회상]",추억 회상
2,"[까페, 잔잔한]",까페 잔잔한
3,"[연말, 눈, 캐럴, 분위기, 따듯, 크리스마스캐럴, 겨울, 노래, 크리스마스, 겨...",연말 눈 캐럴 분위기 따듯 크리스마스캐럴 겨울 노래 크리스마스 겨울왕국 크리스마스 송
4,[댄스],댄스
...,...,...
115066,"[록, 메탈, 밴드, 사운드, 록, 락, 메탈, 메탈, 락, extreme]",록 메탈 밴드 사운드 록 락 메탈 메탈 락 extreme
115067,[렉],렉
115068,"[담시, 가족, 눈물, 그리움, 주인공, 이야기, 사랑, 친구]",담시 가족 눈물 그리움 주인공 이야기 사랑 친구
115069,"[잔잔한, 버스, 퇴근, 버스, Pop, 풍경, 퇴근길]",잔잔한 버스 퇴근 버스 pop 풍경 퇴근길


In [262]:
total_df.to_csv('data/total_nlp.csv', index=False, encoding='utf-8')

### 태그 중에서 빈도 수가 적은거는 지워버리기
### tf_idf를 통해 빈도수 확인

### 이 라이브러리들 확인
- mecap ★
- sentencepiece

### "사용자 사전"을 만들어서 아티스트 이름 같은거는 안나누도록 설정

### 태그를 가지ㅏ고  개수 확인
### 분도포 그리기
### 개수 적은거는 지우기

### 자카드 유사도 
### 공백 기준으로 태그를 나눔
### 공백을 최대한 지움

### 태그 없는 것들은 그냥 없는 상태 그대로 내비두자...